In [1]:
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Dataset, Data
import numpy as np 
import os
from tqdm import tqdm
import copy
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class pMHCDataset(Dataset):
    def __init__(self, root, filename, aaindex, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.filename = filename
        self.aaindex = aaindex
        super(pMHCDataset, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    @property
    def processed_file_names(self):
        """ If these files are found in processed_dir, processing is skipped"""
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()
        return [f'data_{i}.pt' for i in list(self.data.index)]

    def download(self):
        pass##不需要下载
    
    def process(self):
        self.data = pd.read_csv(self.raw_paths[0])
        for index, sample in tqdm(self.data.iterrows(), total=self.data.shape[0]):#tqdm可以显示运行进程
            # Get node features
            node_feats = self._get_node_features(sample["pep"],sample["hla_seq"],self.aaindex)
            edge_index = self._get_edge_index(sample["pep"],sample["hla_seq"])
            label = self._get_labels(sample["type"])
            # Create data object
            data = Data(x=node_feats, edge_index=edge_index, y=label, index=0) 
            torch.save(data, os.path.join(self.processed_dir, f'data_{index}.pt'))

    def _get_node_features(self, pep, HLA, aaindex):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of Nodes, Node Feature size]
        """
        all_seq = pep + HLA
        all_node_feats = []
        for index, aa in enumerate(all_seq):
            node_feats = []
            ##aaindex
            node_feats.extend(aaindex[aa].to_list())
            anchar = [0,len(pep)]
            seq_onehot = [0,0]
            seq_onehot[sum([index >= i for i in anchar])-1] = 1
            node_feats.extend(seq_onehot)
            all_node_feats.append(node_feats)
        all_node_feats = np.asarray(all_node_feats)
        return torch.tensor(all_node_feats)
        
    
    def _get_labels(self, label):
        label = np.asarray([label])
        return torch.tensor(label)
    
    def _get_edge_index(self, pep, hla):
        ##生成边
        nodes = list(range(0,len(pep)+len(hla)))
        edge_index = [[],[]]  
        for i,_ in enumerate(pep):
            nodes_cp = copy.deepcopy(nodes)
            nodes_cp.remove(i)
            edge_index[0].extend([i]*(len(nodes)-1))
            edge_index[1].extend(nodes_cp)
        for i,_ in enumerate(hla):
            i = i + len(pep)
            nodes_cp = copy.deepcopy(nodes)
            nodes_cp.remove(i)
            edge_index[0].extend([i]*(len(nodes)-1))
            edge_index[1].extend(nodes_cp)  
        edge_index = torch.tensor(edge_index)
        return edge_index
    
    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        data = torch.load(os.path.join(self.processed_dir, f'data_{idx}.pt')) 
        return data

In [2]:
aaindex = pd.read_csv("../data/aaindex1_pca.csv")
train_dt = pMHCDataset(root="../data/models/train_data/",
                       filename="train_data_iedb_2.csv",
                       aaindex=aaindex)

In [3]:
##model
from torch_geometric.nn import TransformerConv, TopKPooling, GraphNorm
from torch.nn import Linear, BatchNorm1d, ModuleList, LeakyReLU
from gtrick import random_feature
from gtrick.pyg import VirtualNode
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.nn import MemPooling
from torch.nn import LeakyReLU
class GNN(torch.nn.Module):
    def __init__(self, feature_size, model_params):
        super().__init__()
        
        embedding_size = model_params["model_embedding_size"]
        dense_neurons = model_params["model_dense_neurons"]
        n_heads = model_params["model_heads"]
        n_layers = model_params["model_layers"]
        self.n_layers = n_layers
        self.top_k_every_n = 1
        self.conv_layers = ModuleList([])
        self.transf_layers = ModuleList([])
        self.pooling_layers = ModuleList([])
        self.bn_layers = ModuleList([])
        self.vns = ModuleList()
        self.relu = LeakyReLU()

        # Transformation layer
        self.conv1 = TransformerConv(feature_size, 
                                    embedding_size, 
                                    heads=n_heads,
                                    beta=True) 

        self.transf1 = Linear(embedding_size*n_heads, embedding_size)
        self.bn1 =  GraphNorm(embedding_size)
        # Other layers
        for i in range(n_layers):
            self.conv_layers.append(TransformerConv(embedding_size, 
                                                    embedding_size, 
                                                    heads=n_heads,
                                                    beta=True))

            self.transf_layers.append(Linear(embedding_size*n_heads, embedding_size))
            self.bn_layers.append(GraphNorm(embedding_size))
            self.vns.append(VirtualNode(embedding_size, embedding_size))
            

        # Linear layers
        self.linear1 = Linear(embedding_size, dense_neurons)
        self.linear2 = Linear(dense_neurons, 1)  

    def forward(self, x, edge_index, batch_index, node_index):
        # Initial transformation
        x = self.conv1(x, edge_index)
        x = self.relu(self.transf1(x))
        x = self.bn1(x, batch_index)

        for i in range(self.n_layers):
            x, vx = self.vns[i].update_node_emb(x, edge_index, batch_index)
            x = self.conv_layers[i](x, edge_index)
            x = self.relu(self.transf_layers[i](x))
            x = self.bn_layers[i](x, batch_index)
            vx = self.vns[i].update_vn_emb(x, batch_index, vx)
        
        #x = x[node_index,:]
        # Output block
        x = self.relu(self.linear1(vx))
        x = self.linear2(x)
        return x

In [4]:
def calculate_metrics(y_pred, y_true, epoch, type):
    print(f"{type}, Epoch: {epoch}: ")
    print(f"\n Confusion matrix: \n {confusion_matrix(y_true, y_pred)}")
    print(f"F1 Score: {f1_score(y_true, y_pred)}")
    print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    try:
        roc = roc_auc_score(y_true, y_pred)
        print(f"ROC AUC: {roc}")
    except:
        print(f"ROC AUC: notdefined")
    return prec, rec, roc

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def train_one_epoch(epoch, model, train_loader, optimizer, loss_fn):
    # Enumerate over the data
    all_preds = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for _, batch in enumerate(tqdm(train_loader)): 
        #remove = batch[1]
        target = batch.y
        input_x = batch.x
        edge_index = batch.edge_index
        target = target.to(device)
        input_x = input_x.to(device)
        edge_index = edge_index.to(device)
        batch_index = batch.batch.to(device)
        node_index = batch.index.to(device)
        optimizer.zero_grad() 
        pred = model(input_x.float(), edge_index, batch_index, node_index) 
        # Calculating the loss and gradients
        loss = loss_fn(pred, target.reshape(pred.shape[0],1).float())
        loss.backward()  
        optimizer.step()  
        # Update tracking
        running_loss += loss.item()
        step += 1
        all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_labels.append(target.cpu().detach().numpy())
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    prec, rec, roc = calculate_metrics(all_preds, all_labels, epoch, "train")
    return running_loss/step, prec, rec, roc

def test(epoch, model, test_loader, loss_fn):
    all_preds = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for batch in test_loader:
        target = batch.y
        input_x = batch.x
        edge_index = batch.edge_index
        target = target.to(device)
        input_x = input_x.to(device)
        edge_index = edge_index.to(device)
        batch_index = batch.batch.to(device)
        node_index = batch.index.to(device)
        pred = model(input_x.float(), edge_index, batch_index, node_index) 
        loss = loss_fn(pred, target.reshape(pred.shape[0],1).float())
         # Update tracking
        running_loss += loss.item()
        step += 1
        all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_labels.append(target.cpu().detach().numpy())
        
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    prec, rec, roc = calculate_metrics(all_preds, all_labels, epoch, "test")
    return running_loss/step, prec, rec, roc

# %% Run the training
def run_one_training(params, train_dt, loss_fn, writer):

    # Prepare training
    train_loader = DataLoader(train_dt, batch_size=params["batch_size"], num_workers = 20, pin_memory=True, shuffle=True)
    #test_loader = DataLoader(test_dt, batch_size=params["batch_size"], num_workers = 20, pin_memory=True, shuffle=True)

    # Loading the model
    print("Loading model...")
    model_params = {k: v for k, v in params.items() if k.startswith("model_")}
    model = GNN(feature_size=22, model_params=model_params) 
    model = model.to(device)
    print(f"Number of parameters: {count_parameters(model)}")

    # < 1 increases precision, > 1 recall
    optimizer = torch.optim.Adam(model.parameters(), 
                                lr=params["learning_rate"],
                                weight_decay=params["weight_decay"])
    #scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=params["scheduler_gamma"])

    # Start training
    for epoch in range(36): 
        # Training
        model.train()
        loss, prec, rec, roc = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn)
        print(f"Epoch {epoch} | Train Loss {loss}")
        writer.add_scalar("Train/Loss", loss, epoch)
        # Testing
        #if epoch % 5 == 0:
        #    model.eval()
        #    loss, prec, rec, roc = test(epoch, model, test_loader, loss_fn)
        #    print(f"Epoch {epoch} | Test Loss {loss}")
    return model

In [5]:
import numpy as np
import pandas as pd
import torch
from torch.utils import data
from torch_geometric.loader import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from tqdm import tqdm
HYPERPARAMETERS = {
    "batch_size": 8,
    "weight_decay": 0.00001,
    "learning_rate": 0.0001,
    "model_embedding_size": 64, 
    "model_dense_neurons": 32,
    "model_heads":3,
    "model_layers":3
}
writer = SummaryWriter(f"/root/tf-logs/gnn_neo_loss")
loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor(1.4))
model_last = run_one_training(HYPERPARAMETERS,train_dt,loss_fn,writer)

Loading model...
Number of parameters: 273089


100%|██████████| 1052/1052 [00:57<00:00, 18.16it/s]


train, Epoch: 0: 

 Confusion matrix: 
 [[3040 1905]
 [1098 2369]]
F1 Score: 0.6120656245963054
Accuracy: 0.6430099857346647
Precision: 0.5542817033224146
Recall: 0.6832996827228152
ROC AUC: 0.6490310344857757
Epoch 0 | Train Loss 0.731081186308607


100%|██████████| 1052/1052 [00:54<00:00, 19.28it/s]


train, Epoch: 1: 

 Confusion matrix: 
 [[3232 1713]
 [ 751 2716]]
F1 Score: 0.6879432624113475
Accuracy: 0.7070851165002378
Precision: 0.6132309776473245
Recall: 0.7833862128641477
ROC AUC: 0.7184878485958758
Epoch 1 | Train Loss 0.6584465550435813


...

train, Epoch: 33: 

 Confusion matrix: 
 [[3739 1206]
 [ 728 2739]]
F1 Score: 0.7390717754991905
Accuracy: 0.7700903471231574
Precision: 0.694296577946768
Recall: 0.7900201903663109
ROC AUC: 0.7730687402792121
Epoch 33 | Train Loss 0.559595361989374


100%|██████████| 1052/1052 [00:56<00:00, 18.71it/s]


train, Epoch: 34: 

 Confusion matrix: 
 [[3735 1210]
 [ 679 2788]]
F1 Score: 0.74695244474213
Accuracy: 0.7754398478364242
Precision: 0.6973486743371686
Recall: 0.804153446783963
ROC AUC: 0.7797309195497166
Epoch 34 | Train Loss 0.552113721791669


100%|██████████| 1052/1052 [00:56<00:00, 18.67it/s]

train, Epoch: 35: 

 Confusion matrix: 
 [[3747 1198]
 [ 675 2792]]
F1 Score: 0.7488266058736757
Accuracy: 0.7773418925344746
Precision: 0.699749373433584
Recall: 0.8053071820017306
ROC AUC: 0.7815211339735649
Epoch 35 | Train Loss 0.5449835354242715


In [7]:
torch.save(model_last.state_dict(), "../data/models/gnn_full_model_ext.pt")

In [14]:
###预测
dt = pd.read_csv("../data/tesla_val.csv")
val_dt = pMHCDataset(root="../data/models/val_data/",
                       filename="db_pre_dt.csv",
                       aaindex=aaindex)

Processing...
100%|██████████| 1438/1438 [00:05<00:00, 260.94it/s]
Done!


In [15]:
data_loader = DataLoader(val_dt, batch_size=64, shuffle=False)
all_preds = []
all_labels = []
for _, batch in enumerate(tqdm(data_loader)):
    target = batch.y
    input_x = batch.x
    edge_index = batch.edge_index
    target = target.to(device)
    input_x = input_x.to(device)
    edge_index = edge_index.to(device)
    batch_index = batch.batch.to(device)
    node_index = batch.index.to(device)
    model_last.eval()
    pred = model_last(input_x.float(), edge_index, batch_index, node_index) 
    all_preds.append(torch.sigmoid(pred).cpu().detach().numpy())
    all_labels.append(target.cpu().detach().numpy())
all_preds = np.concatenate(all_preds).ravel()
all_labels = np.concatenate(all_labels).ravel()
pred = pd.DataFrame({"all_preds":all_preds,"all_labels":all_labels})
pred.to_csv("../data/db_pred_gnn.csv")

100%|██████████| 23/23 [00:01<00:00, 17.01it/s]


In [16]:
torch.save(model_last, "../data/models/last_model.pt")